In [5]:
import cv2
import numpy as np
import os
from inference_sdk import InferenceHTTPClient
from glob import glob
from tqdm import tqdm

In [6]:
# Initialize the Roboflow inference client
CLIENT = InferenceHTTPClient(
    api_url="https://outline.roboflow.com",
    api_key="7Oj4lP3s5DlHk2Syfc6G"
)
MODEL_ID = "data_tonghop/2"

In [7]:
# Input and output setup
input_folder = "./batch1"  # Replace with your video folder path
output_base_dir = "./batch1/output_segmented"
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

# Supported video extensions
video_extensions = ["*.mp4", "*.avi", "*.mov"]

In [8]:
# Process each video
for ext in video_extensions:
    for video_path in glob(os.path.join(input_folder, ext)):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        output_dir = os.path.join(output_base_dir, video_name)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Open video
        cap = cv2.VideoCapture(video_path)

        # Verify video dimensions
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if width != 1280 or height != 480:
            print(f"Skipping {video_name}: Dimensions must be 1280x480, got {width}x{height}")
            cap.release()
            continue

        # Get total frame count for progress bar
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_count = 0
        temp_rgb_path = f"temp_rgb_{video_name}.jpg"

        # Process frames with progress bar
        with tqdm(total=total_frames, desc=f"Processing {video_name}", unit="frame") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_count += 1

                # Split frame into depth (left) and RGB (right)
                depth_frame = frame[:, :640, :]  # Left 640x480
                rgb_frame = frame[:, 640:, :]    # Right 640x480

                # Save RGB frame temporarily for inference
                cv2.imwrite(temp_rgb_path, rgb_frame)

                # Run inference on RGB frame
                try:
                    predictions = CLIENT.infer(temp_rgb_path, model_id=MODEL_ID)
                except Exception as e:
                    print(f"Inference failed for {video_name}, frame {frame_count}: {e}")
                    pbar.update(1)
                    continue

                # Process predictions to extract pig masks
                current_masks = []
                for pred in predictions.get("predictions", []):
                    if pred.get("class") == "pig":  # Adjust if class name differs
                        mask = np.zeros((480, 640), dtype=np.uint8)
                        if "points" in pred:  # If model returns polygon
                            points = np.array([[p["x"], p["y"]] for p in pred["points"]], dtype=np.int32)
                            cv2.fillPoly(mask, [points], 255)
                        elif "mask" in pred:  # If model returns bitmap mask
                            mask = np.array(pred["mask"], dtype=np.uint8)
                            if mask.shape != (480, 640):
                                mask = cv2.resize(mask, (640, 480), interpolation=cv2.INTER_NEAREST)
                        current_masks.append(mask)

                # Apply masks to segment RGB and depth frames
                for i, mask in enumerate(current_masks):
                    # Ensure mask is binary
                    mask = (mask > 0).astype(np.uint8)

                    # Segment RGB frame
                    segmented_rgb = cv2.bitwise_and(rgb_frame, rgb_frame, mask=mask)

                    # Segment depth frame
                    if depth_frame.shape[2] == 3:  # If depth is RGB-encoded
                        segmented_depth = cv2.bitwise_and(depth_frame, depth_frame, mask=mask)
                    else:  # If depth is grayscale
                        segmented_depth = cv2.bitwise_and(depth_frame, depth_frame, mask=mask)
                        segmented_depth = cv2.cvtColor(segmented_depth, cv2.COLOR_GRAY2BGR)  # Convert for PNG

                    # Save every 5 frames
                    if frame_count % 5 == 0:
                        cv2.imwrite(os.path.join(output_dir, f"frame_{frame_count}_rgb_crop_{i}.png"), segmented_rgb)
                        cv2.imwrite(os.path.join(output_dir, f"frame_{frame_count}_depth_crop_{i}.png"), segmented_depth)

                # Update progress bar
                pbar.update(1)

        # Clean up
        cap.release()
        if os.path.exists(temp_rgb_path):
            os.remove(temp_rgb_path)

Processing depth_rgb_recording(8): 100%|██████████| 667/667 [13:59<00:00,  1.26s/frame]
